<a href="https://colab.research.google.com/github/Hareesh292/Data_Analysis1/blob/main/dataAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Load CSV Data

In [5]:
!pip install pandas openpyxl fuzzywuzzy python-Levenshtein


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 44.9 MB/s eta 0:00:00


In [7]:
from google.colab import files
uploaded = files.upload()


Saving [V2] Data Alchemist - Sample Data - Clients 1.csv to [V2] Data Alchemist - Sample Data - Clients 1 (1).csv


In [1]:
import pandas as pd

clients_df = pd.read_csv('[V2] Data Alchemist - Sample Data - Clients 1.csv')
clients_df.head()


,ClientID,ClientName,PriorityLevel,RequestedTaskIDs,GroupTag,AttributesJSON
0,C1,Acme Corp,3,"T17,T27,T33,T31,T20,T3,T32,T26",GroupA,"{""location"":""New York"",""budget"":100000}"
1,C2,Globex Inc,1,"T35,T39,T10,T17,T46",GroupB,"{""message"":""ensure deliverables align with pro..."
2,C3,Initech,4,"T28,T7,T23,T40,T41,T31,T36,T21,T50,T14",GroupA,"{""sla"":""24h"",""vip"":true}"
3,C4,Umbrella Co,5,"T22,T1,T43,T21,T32,T41,T16,T5,T40,T13",GroupC,"{""message"":""budget approved pending CFO review..."
4,C5,Stark Industries,2,"T36,T32,T8,T47,T7,T19,T20",GroupB,"{""notes"":""rush order"",""budget"":200000}"


Validate Columns

In [2]:
expected_client_columns = [
    "ClientID", "ClientName", "PriorityLevel", "RequestedTaskIDs", "GroupTag", "AttributesJSON"
]

def validate_columns(df, expected_columns):
    missing_cols = [col for col in expected_columns if col not in df.columns]
    extra_cols = [col for col in df.columns if col not in expected_columns]
    return missing_cols, extra_cols

missing, extra = validate_columns(clients_df, expected_client_columns)
print("Missing columns:", missing)
print("Extra columns:", extra)

Missing columns: []
Extra columns: []


Validate Data (Priority,JSON,TaskIDs Formate)

In [3]:
import json

def validate_clients_data(df):
    errors = []

    for idx, row in df.iterrows():
        if not 1 <= row['PriorityLevel'] <= 5:
            errors.append(f"Row {idx}: Invalid PriorityLevel - {row['PriorityLevel']}")

        if not isinstance(row['RequestedTaskIDs'], str):
            errors.append(f"Row {idx}: RequestedTaskIDs is not a string")

        try:
            json.loads(row['AttributesJSON'])
        except:
            errors.append(f"Row {idx}: Malformed JSON in AttributesJSON")

    return errors

client_errors = validate_clients_data(clients_df)
print("\n".join(client_errors) if client_errors else "No validation issues.")


No validation issues.


AI -style column Name mapping

In [8]:
from fuzzywuzzy import fuzz, process

def fuzzy_column_mapper(df_cols, expected_cols):
    mapping = {}
    for col in expected_cols:
        match, score = process.extractOne(col, df_cols)
        mapping[col] = match if score >= 80 else None
    return mapping

fuzzy_mapping = fuzzy_column_mapper(clients_df.columns.tolist(), expected_client_columns)
print(fuzzy_mapping)


{'ClientID': 'ClientID', 'ClientName': 'ClientName', 'PriorityLevel': 'PriorityLevel', 'RequestedTaskIDs': 'RequestedTaskIDs', 'GroupTag': 'GroupTag', 'AttributesJSON': 'AttributesJSON'}


Create a Sample Rule (Basic Co-run Rule Generation)

In [9]:
# Let's define co-run for TaskIDs
def create_co_run_rule(task_ids):
    return {
        "type": "coRun",
        "tasks": task_ids
    }

# Example usage:
co_run_rule = create_co_run_rule(["T001", "T003"])
print(co_run_rule)


{'type': 'coRun', 'tasks': ['T001', 'T003']}


Export Cleaned Data and Rules

In [11]:
# Save cleaned client data
clients_df.to_csv("cleaned_clients.csv", index=False)

# Save rules.json
import json

rules = [co_run_rule]  # Add more as needed
with open("rules.json", "w") as f:
    json.dump(rules, f, indent=2)

# Download files
files.download("cleaned_clients.csv")
files.download("rules.json")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>